<a href="https://colab.research.google.com/github/Aligreu/LLMs/blob/main/microsoft_phi_3_instruct_3_8b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# prompt: import notebook_login from huggingface
from huggingface_hub import notebook_login

notebook_login()

In [16]:
from IPython.display import display, Markdown

In [3]:
# prompt: load phi model from transformers package

from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")
model = AutoModelForCausalLM.from_pretrained("microsoft/Phi-3-mini-4k-instruct",
                                             torch_dtype=torch.float16)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y
The repository for microsoft/Phi-3-mini-4k-instruct contains custom code which must be executed to correctly load the model. You can inspect the repository content at https://hf.co/microsoft/Phi-3-mini-4k-instruct.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


model.safetensors.index.json:   0%|          | 0.00/16.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

In [4]:
model.to("cuda")

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

In [22]:
text = '''<|user|> write a function in python that implement linear regression with xgboost<|end|>\n<|assistant|>\n'''

ids = tokenizer(text, return_tensors='pt').to("cuda")
output = model.generate(**ids, max_new_tokens=1000)
display(Markdown(tokenizer.decode(output[0], skip_special_tokens=True)))

write a function in python that implement linear regression with xgboost To implement linear regression using XGBoost in Python, you can follow these steps. XGBoost is primarily a gradient boosting framework, but it can be used to approximate linear regression by minimizing the loss function. However, for a direct linear regression approach, we'll use XGBoost's `reg:linear` objective.

First, ensure you have XGBoost installed. If not, you can install it using pip:

```bash
pip install xgboost
```

Here's a Python function that implements linear regression using XGBoost:

```python
import xgboost as xgb
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def linear_regression_xgboost(X, y):
    """
    Implements linear regression using XGBoost.

    Parameters:
    - X: numpy array or pandas DataFrame, features.
    - y: numpy array or pandas Series, target variable.

    Returns:
    - model: trained XGBoost model.
    - mse: Mean Squared Error on the test set.
    """
    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Convert data into DMatrix format required by XGBoost
    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)

    # Set up parameters for XGBoost
    params = {
        'objective': 'reg:linear',  # Linear Regression objective
        'eval_metric': 'rmse',     # Root Mean Squared Error as the evaluation metric
        'max_depth': 6,            # Maximum tree depth
        'eta': 0.1,                # Learning rate
        'subsample': 0.8,          # Subsample ratio of the training instances
        'colsample_bytree': 0.8,   # Subsample ratio of columns when constructing each tree
        'silent': 1,               # Be silent during training
        'n_estimators': 100        # Number of boosting rounds
    }

    # Train the model
    model = xgb.train(params, dtrain, num_boost_round=params['n_estimators'])

    # Predict on the test set
    y_pred = model.predict(dtest)

    # Calculate Mean Squared Error (MSE)
    mse = mean_squared_error(y_test, y_pred)

    return model, mse

# Example usage
# Assuming X and y are already defined and preprocessed
# X = ...
# y = ...
# model, mse = linear_regression_xgboost(X, y)
# print(f"Trained model: {model}")
# print(f"Mean Squared Error: {mse}")
```

This function trains an XGBoost model with a linear regression objective and evaluates its performance on a test set using the Root Mean Squared Error (RMSE). Adjust the parameters (`params`) as needed to fine-tune the model's performance.

In [21]:
text = '''<|user|> suggest 10 text books to learn machine learning\n<|end|>\n<|assistant|>\n'''

ids = tokenizer(text, return_tensors='pt').to("cuda")

output = model.generate(**ids, max_new_tokens=1000)

display(Markdown(tokenizer.decode(output[0], skip_special_tokens=True)))

suggest 10 text books to learn machine learning
 Here are 10 textbooks that are highly recommended for learning machine learning:

1. "Pattern Recognition and Machine Learning" by Christopher M. Bishop
2. "The Hundred-Page Machine Learning Book" by Andriy Burkov
3. "Machine Learning: A Probabilistic Perspective" by Kevin P. Murphy
4. "Introduction to Machine Learning with Python" by Andreas Müller & Sarah Guido
5. "Machine Learning: A Bayesian and Optimization Perspective" by Kevin P. Murphy
6. "Deep Learning" by Ian Goodfellow, Yoshua Bengio, and Aaron Courville
7. "Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow" by Aurélien Géron
8. "Python Machine Learning" by Sebastian Raschka
9. "Machine Learning: A Probabilistic Perspective" by Kevin P. Murphy (second edition)
10. "The Elements of Statistical Learning" by Trevor Hastie, Robert Tibshirani, and Jerome Friedman

These books cover a wide range of topics in machine learning, from foundational concepts to advanced techniques, and are written by reputable authors in the field. They are suitable for beginners, intermediate learners, and advanced practitioners alike.

In [23]:
text = '''<|user|> write a multiprocessing function in python that multipkly a seuence of 1000 number by 10 and uses 8 cores

<|end|>\n<|assistant|>\n'''

ids = tokenizer(text, return_tensors='pt').to("cuda")
output = model.generate(**ids, max_new_tokens=1000)

print(tokenizer.decode(output[0], skip_special_tokens=True))

write a multiprocessing function in python that multipkly a seuence of 1000 number by 10 and uses 8 cores

 To achieve multiprocessing in Python for multiplying a sequence of 1,000 numbers by 10 using 8 cores, you can use the `multiprocessing` module. Below is a Python function that demonstrates this:


```python

import multiprocessing

import numpy as np


def multiply_by_ten(number):

    return number * 10


def multiprocessing_multiply(numbers, num_cores=8):

    # Create a pool of workers

    pool = multiprocessing.Pool(processes=num_cores)


    # Split the list of numbers into chunks for each process

    chunk_size = len(numbers) // num_cores

    chunks = [numbers[i:i + chunk_size] for i in range(0, len(numbers), chunk_size)]


    # Map the multiply_by_ten function to each chunk

    results = pool.map(multiply_by_ten, chunks)


    # Close the pool and wait for the work to finish

    pool.close()

    pool.join()


    return results


# Generate a sequence of 1,000 numbe